####  Extending P0 with Bitwise Set Operations (WASM)

In [1]:
def runwasm(wasmfile):
    from IPython.display import display, Javascript
    display(Javascript("""
    const params = {
        P0lib: {
            write: i => element.append(i + ' '),
            writeln: () => element.append(document.createElement('br')),
            read: () => window.prompt()
        }
    }
    
    var wasmByteString = \"""" + str(open(wasmfile, "rb").read()) + """\"; // pass the wasm file to JavaScript as byte string
    wasmByteString = wasmByteString.substring(2, wasmByteString.length - 1); // remove the byte literals b'...'
    const wasmArrayBuffer = new Uint8Array(wasmByteString.length); // convert the binary string to ArrayBuffer
    for (let i = 0; i < wasmByteString.length; i++)
      wasmArrayBuffer[i] = wasmByteString.charCodeAt(i);
    
    WebAssembly.compile(wasmArrayBuffer.buffer) // compile (sharable) code
        .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
        // .then(instance => instance.exports.program()); // run the main program; not needed if a start function is specified
     """))

In [2]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i, end=' ')
    def writeln(s): print()
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [3]:
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i, end=' ')
    def writeln(): print()
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

In [4]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

Extend P0 with two binary operators, `∉` (Unicode U+2209, not an element of) and `∖` (Unicode U+2216, set minus), defined by:
* `i ∉ s ≡ ¬(i ∈ s)`, where `i` is an integer and `s` is a set 
* `s ∖ t = s ∩ ∁t`, where `s`, `t` are sets

The following P0 program illustrates the use of these operators; you can use it to test your implementation:

In [5]:
!wat2wasm primes.wat

In [6]:
compileString("""
const N = 32
const R = 5 // ⌊√N⌋
type S = set [0 .. N - 1]
procedure eratosthenes() → (p: S)
    var i, j: integer
        p := ∁ {} // set of all integers 
        i := 2;
        while i ≤ R do
            if i ∈ p then
                j := i × i
                while j < N do
                    p, j := p ∖ {j}, j + i
            i := i + 1
program primes
    var p: S
    var j: integer
        p ← eratosthenes()
        j := 2 // print all primes up to N
        while j < N do
            if j ∈ p then write(j)
            j := j + 1
        writeln(); j := 2 // print all non-primes up to N
        while j < N do
            if j ∉ p then write(j)
            j := j + 1
""", 'primes.wat', target = 'wat')

In [7]:
runpywasm("primes.wasm")

2 3 5 7 11 13 17 19 23 29 31 
4 6 8 9 10 12 14 15 16 18 20 21 22 24 25 26 27 28 30 

The output is supposed to be:
```
2 3 5 7 11 13 17 19 23 29 31 
4 6 8 9 10 12 14 15 16 18 20 21 22 24 25 26 27 28 30
```

##### Part A (Extending the Scanner) [4 points]

- In `SC.ipynb`, introduce new integer constants `NOTELEMENT` and `DIFFERENCE` for `∉` and `∖`.
- Extend the production of `symbol` in the text cell above `getSym()` to include  `∉` and `∖`.
- Extend `getSym()` to return `NOTELEMENT` and `DIFFERENCE` when recognizing  `∉` and `∖`.

The cell below serves only for grading and testing.

In [8]:
import SC

def scanString(src):
    SC.init(src); syms = [(SC.sym, SC.val)]
    while SC.sym != SC.EOF:
        SC.getSym()
        syms.append((SC.sym, SC.val))
    return syms

assert scanString('5 ∉ {3}') == [(SC.NUMBER, 5), (SC.NOTELEMENT, 5), (SC.LBRACE, 5), 
                                 (SC.NUMBER, 3), (SC.RBRACE, 3), (SC.EOF, 3)]
assert scanString('s ∖ t') == [(SC.IDENT, 's'), (SC.DIFFERENCE, 's'), (SC.IDENT, 't'), (SC.EOF, 't')]
    

##### Part B (Extending the Parser) [8 points]

1. In `P0.ipynb`, extend the imports with `NOTELEMENT` and `DIFFERENCE`.
2. Extend the grammar at two places, in the section "The P0 Grammar" and before the parsing procedures for each nonterminal: `∖` has to bind as tight as `∩` and `∉` has to bind as tight as `∈`. That is, `a ∉ b ∖ c ∪ d` is parsed like `a ∉ ((b ∖ c) ∪ d)`.
3. If the first sets need to be modified, modify them.
4. Extend the parsing procedures accordingly. Add type-checking: when parsing `a ∉ b`, if `a` is not an integer, the error `bad type` should be generated; if `b` is not a set, the error `set expected` should be generated. When parsing `(a ∖ b)`, the error `bad type` should be generated if not both `a` and `b` are sets.
5. In `CGast.ipynb`, extend the imports with `NOTELEMENT` and `DIFFERENCE`.
6. Extend method `__str(self)__` of class `BinaryOp` such that `∉` is returned for `NOTELEMENT` and `∖` is returned for `DIFFERENCE`.

The cell below serves only for grading and testing.

In [9]:
assert compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var c: boolean
    c := (2 ∈ a) or (2 ∉ a)
    a := ∁ a ∩ b ∖ a ∪ b
""", target='ast') == """\
seq
  :=
    Var(name = c, lev = 1, tp = <class 'ST.Bool'>)
    or
      ∈
        Const(name = , tp = <class 'ST.Int'>, val = 2)
        Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
      ∉
        Const(name = , tp = <class 'ST.Int'>, val = 2)
        Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
  :=
    Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
    ∪
      ∖
        ∩
          ∁
            Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
          Var(name = b, lev = 1, tp = Set(lower = 1, length = 5))
        Var(name = a, lev = 1, tp = Set(lower = 1, length = 5))
      Var(name = b, lev = 1, tp = Set(lower = 1, length = 5))"""

The following programs should all produce error messages:

In [10]:
try:
    compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var c: boolean
    c := a ∉ b
""", target='ast')
    raise
except Exception as e:
    assert str(e) == "line 5 pos 14 bad type"

In [11]:
try:
    compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var c: boolean
    c := 2 ∉ c
""", target='ast')
    raise
except Exception as e:
    assert str(e) == "line 5 pos 14 set expected"

In [12]:
try:
    compileString("""
program bitsets
  var a, b: set [1 .. 5]
  var c: boolean
    a := c ∖ b
""", target='ast')
    raise
except Exception as e:
    assert str(e) == "line 5 pos 14 bad type"

In [13]:
try:
    compileString("""
program bitsets
  var a, b: set [1 .. 5]
    a := b ∖ 3
""", target='ast')
    raise
except Exception as e:
    assert str(e) == "line 4 pos 14 bad type"

##### Part C (Extending the Code Generator) [8 points]

In `CGwat.ipynb`, extend `genUnaryOp` and `genBinaryOp` to generate code for `∉` and `∖`.

The cell below serves only for grading and testing. *Hint:* when generating code for `a ∖ b` in `genBinaryOp`, load first `b` on the stack, complement it, then load `a` on the stack and bitwise-and the top two elements on the stack.

In [14]:
import nbimporter; nbimporter.options["only_defs"] = False
import P0

compileString("""
program bitsets
  var a, b: set [1 .. 11]
  var i: integer
    i, a := 1, {3, 5, 7}
    if i ∉ a then a := a ∖ {7, 9}
    while i < 12 do
      if i ∈ a then write(i)
      i := i + 1
""", 'bitsets.wat', target = 'wat')

assert open('bitsets.wat').read() == """\
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $a i32)
(local $b i32)
(local $i i32)
(local $0 i32)
i32.const 1
i32.const 3
local.set $0
i32.const 1
local.get $0
i32.shl
i32.const 5
local.set $0
i32.const 1
local.get $0
i32.shl
i32.or
i32.const 7
local.set $0
i32.const 1
local.get $0
i32.shl
i32.or
local.set $a
local.set $i
local.get $i
local.set $0
i32.const 1
local.get $0
i32.shl
local.get $a
i32.and
i32.eqz
if
i32.const 7
local.set $0
i32.const 1
local.get $0
i32.shl
i32.const 9
local.set $0
i32.const 1
local.get $0
i32.shl
i32.or
i32.const 0xffe
i32.xor
local.get $a
i32.and
local.set $a
end
loop
local.get $i
i32.const 12
i32.lt_s
if
local.get $i
local.set $0
i32.const 1
local.get $0
i32.shl
local.get $a
i32.and
if
local.get $i
call $write
end
local.get $i
i32.const 1
i32.add
local.set $i
br 1
end
end
)
(memory 1)
(start $program)
)"""


The first if-statement should translate to:
```
local.get $i
local.set $0
i32.const 1
local.get $0
i32.shl
local.get $a
i32.and
i32.eqz
if
i32.const 7
local.set $0
i32.const 1
local.get $0
i32.shl
i32.const 9
local.set $0
i32.const 1
local.get $0
i32.shl
i32.or
i32.const 0xffe
i32.xor
local.get $a
i32.and
local.set $a
end
```

Running the program should print `3 5`:

In [15]:
!wat2wasm bitsets.wat

In [16]:
runpywasm("bitsets.wasm")

3 5 

##### Part D (Evaluating the Implementation) [4 points].

The task is to compare the efficiency of 4 implementations of the Sieve of Eratosthenes with sets:
- P0 using pywasm, the Python interpreter of WebAssembly,
- P0 using the JavaScript host of WebAssembly in your web browser,
- Python using the standard CPython implementation,
- Java using the standard Oracle JVM.

The P0, Python, and Java implementations are already provided. To keep it simple, the [Jupyter time cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) is used rather than printing the start and end time within the programs. That command measures the *wall clock time*. This means that the time for loading and compilation is included in the measured time. We, therefore, aim at execution times above 1 second to make the time for loading and compiling negligible. For this, the Sieve of Eratosthenes is repeatedly run. The times will vary with each run. Longer times typically result from an unrelated CPU load and should be disregarded. The best is to run the code at times of the day when there is as little interference as possible. Vary the number of repetitions. Document your observations and explain!  

After running all of the 4 implemenetations of the Sieve of Eratosthenes multiple times on 1000 repetitions, I got the following results. On average:
- P0 using pywasm had CPU times: user 885ms, sys: 0ns-3ms,  total: 885ms, Wall time: 890ms
- P0 using JS host had CPU times: user 1-2ms, sys: 0ns-2ms, total: 2.2ms, Wall time: 1.6ms
- Python had CPU times: user 5ms, sys: 1-8ms, total: 10-11ms, Wall time: 5.3ms
- JS had CPU times: user 1-4ms, sys: 6-7ms, total: 12-13ms, Wall time: 215ms

From all of this data, it is evident that P0 using pywasm is extremely slow, and most definitely the worst out of the 4 implementations given.
It is also evident that Java using the standard Oracle JVM is the second worst implementation.
When observing the differences between the two fastest implementations (P0 using JS and Python using the standard CPython implementation), it can be seen that P0 using JS is faster than pure Python.
Thus, in order of fastest to slowest implementation, we can give the following ordering: P0 using the JavaScript host of WebAssembly in your web browser, then Python using the standard CPython implementation, then Java using the standard Oracle JVM, and finally, P0 using pywasm, the Python interpreter of WebAssembly. Or in short, P0 JS -> PY -> JS -> P0 PY.
After testing it on a multitude of repetitions, this pattern is evident in all cases (10000 repetitions, 100000, etc).

In [17]:
compileString("""
const N = 32
const R = 5 // ⌊√N⌋
type S = set [2 .. 31]
procedure eratosthenes() → (p: S)
    var i, j: integer
        p := ∁ {} // set of all integers 
        i := 2;
        while i ≤ R do
            if i ∈ p then
                j := i × i
                while j < N do
                    p, j := p ∖ {j}, j + i
            i := i + 1
program repeatprimes
    var p: S
    var j: integer
        j := 0
        while j < 1000 do
            p ← eratosthenes(); j := j + 1
""", 'repeatprimes.wat', target = 'wat')

In [18]:
!wat2wasm repeatprimes.wat

In [19]:
time runpywasm("repeatprimes.wasm")

CPU times: user 912 ms, sys: 3.28 ms, total: 916 ms
Wall time: 921 ms


In [20]:
time runwasm("repeatprimes.wasm")

<IPython.core.display.Javascript object>

CPU times: user 5.45 ms, sys: 3.75 ms, total: 9.21 ms
Wall time: 8.43 ms


In [21]:
time runwasmer("repeatprimes.wasm")

CPU times: user 25.7 ms, sys: 8.93 ms, total: 34.6 ms
Wall time: 12.1 ms


In [22]:
def eratosthenes():
    N, R = 32, 5
    p = {i for i in range(2, N)}
    for i in range(2, R):
        if i in p:
            for j in range(i * i, N, i): p -= {j}
    return p

def repeatprimes():
    for _ in range(1000): eratosthenes()

In [23]:
time repeatprimes()

CPU times: user 5.44 ms, sys: 206 µs, total: 5.65 ms
Wall time: 5.57 ms


In [24]:
%%writefile repeatprimes.java
import java.util.*;
class RepeatPrimes {
    static final int N = 32;
    static final int R = 5;
    static Set<Integer> eratosthenes() {
        Set<Integer> p = new HashSet<Integer>();
        for (int i = 2; i < N; i++) p.add(i);
        for (int i = 2; i < R; i++)
            if (p.contains(i))
                for (int j = i * i; j < N; j = j + i)
                    p.remove(j);
        return p;
    }
    public static void main(String[] args) {
        for (int i = 0; i < 1000; i++) eratosthenes();
    }
}

Overwriting repeatprimes.java


In [25]:
!javac repeatprimes.java

In [26]:
time !java RepeatPrimes

CPU times: user 5.28 ms, sys: 5.15 ms, total: 10.4 ms
Wall time: 217 ms


In [27]:
import nbimporter
nbimporter.options["only_defs"] = False
from SC import TIMES, DIV, MOD, PLUS, MINUS, AND, OR, EQ, NE, LT, GT, LE, GE, \
     NOT, CARD, COMPLEMENT, UNION, INTERSECTION, ELEMENT, NOTELEMENT, DIFFERENCE, SUBSET, SUPERSET, \
     mark
from ST import indent, Bool

class UnaryOp:
    def __init__(self, op, arg):
        self.tp, self.op, self.arg = arg.tp, op, arg
    def __str__(self):
        o = '+' if self.op == PLUS else \
            '-' if self.op == MINUS else \
            '¬' if self.op == NOT else \
            '#' if self.op == CARD else \
            '∁' if self.op == COMPLEMENT else 'op?'
        return o + '\n' + indent(self.arg)

class BinaryOp:
    def __init__(self, op, tp, left, right):
        self.tp, self.op, self.left, self.right = tp, op, left, right
    def __str__(self):
        o = '×' if self.op == TIMES else \
            'div' if self.op == DIV else \
            'mod' if self.op == MOD else \
            'and' if self.op == AND else \
            '+' if self.op == PLUS else \
            '-' if self.op == MINUS else \
            'or' if self.op == OR else \
            '=' if self.op == EQ else \
            '≠' if self.op == NE else \
            '<' if self.op == LT else \
            '>' if self.op == GT else \
            '≤' if self.op == LE else \
            '≥' if self.op == GE else \
            '∪' if self.op == UNION else \
            '∩' if self.op == INTERSECTION else \
            '∈' if self.op == ELEMENT else \
            '∉' if self.op == NOTELEMENT else \
            '∖' if self.op == DIFFERENCE else \
            '⊆' if self.op == SUBSET else \
            '⊇' if self.op == SUPERSET else 'op?'
        return o + '\n' + indent(self.left) + '\n' + indent(self.right)

class Assignment:
    def __init__(self, left, right):
        self.left, self.right = left, right
    def __str__(self):
        return ':=\n' + indent(self.left) + '\n' + indent(self.right)

class Call:
    def __init__(self, res, ident, param):
        self.res, self.ident, self.param = res, ident, param
    def __str__(self):
        return 'call ' + (str(self.res) if self.res else '') + ' ' + \
               str(self.ident) + ('\n' if len(self.param) > 0 else '') + \
               indent('\n'.join([str(x) for x in self.param]))

class Seq:
    def __init__(self, first, second):
        self.first, self.second = first, second
    def __str__(self):
        return 'seq\n' + indent(self.first) + '\n' + indent(self.second)

class IfThen:
    def __init__(self, cond, th):
        self.cond, self.th = cond, th
    def __str__(self):
        return 'ifthen\n' + indent(self.cond) + '\n' + indent(self.th)

class IfElse:
    def __init__(self, cond, th, el):
        self.cond, self.th, self.el = cond, th, el
    def __str__(self):
        return 'ifelse\n' + indent(self.cond) + '\n' + indent(self.th) + \
               '\n' + indent(self.el)

class While:
    def __init__(self, cond, bd):
        self.cond, self.bd = cond, bd
    def __str__(self):
        return 'while\n' + indent(self.cond) + '\n' + indent(self.bd)

class ArrayIndexing:
    def __init__(self, arr, ind):
        self.tp, self.arr, self.ind = arr.tp.base, arr, ind
    def __str__(self):
        return str(self.arr) + '[]\n '+ indent(self.ind)

class FieldSelection:
    def __init__(self, rec, fld):
        self.tp, self.rec, self.fld = fld.tp, rec, fld
#        self.tp, self.rec, self.fld = rec.tp.fields[fld].tp, rec, fld
    def __str__(self):
        return str(self.rec) + '.' + str(self.fld)

# public functions

def genBool(b):
    b.size = 4; return b

def genInt(i):
    i.size = 4; return i

def genRec(r):
    """Assuming r is Record, determine fields offsets and the record size"""
    s = 0
    for f in r.fields:
        f.offset, s = s, s + f.tp.size
    r.size = s
    return r

def genArray(a):
    """Assuming r is Array, determine its size"""
    # adds size
    a.size = a.length * a.base.size
    return a

def genSet(s):
    s.size = 4; return s

def genGlobalVars(sc, start):
    pass

def genLocalVars(sc, start):
    pass

def genProgStart():
    pass

def genProgEntry(ident):
    pass

def genProgExit(x):
    return x

def genProcStart():
    pass

def genProcEntry(ident, parsize, localsize):
    pass

def genProcExit(x, parsize, localsize):
    pass

def genActualPara(ap, fp, n):
    pass

def genSelect(x, f):
    # x.f, assuming f is ST.Field
    return FieldSelection(x, f)

def genIndex(x, y):
    # x[y], assuming x is ST.Var, x.tp is ST.Array, y is Const or Reg integer
    return ArrayIndexing(x, y)

def genVar(x):
    # assuming x is ST.Var, ST.Ref, ST.Const
    return x

def genConst(x):
    return x

def genUnaryOp(op, x):
    return UnaryOp(op, x) if op in \
           {PLUS, MINUS, NOT, CARD, COMPLEMENT} else x

def genBinaryOp(op, x, y):
    return BinaryOp(op, x.tp, x, y)

def genRelation(op, x, y):
    return BinaryOp(op, Bool, x, y)

def genLeftAssign(x):
    return x

def genRightAssign(x):
    return x

def genAssign(x, y):
    return Assignment(x, y)

def genActualPara(ap, fp, n):
    return ap

def genCall(r, pr, ap):
    return Call(r, pr.name, ap)

def genRead(x):
    return Call(x, 'read', [])

def genWrite(x):
    return Call(None, 'write', [x])

def genWriteln():
    return Call(None, 'writeln', [])

def genSeq(x, y):
    return Seq(x, y)

def genThen(x):
    return x

def genIfThen(x, y):
    return IfThen(x, y)

def genElse(x, y):
    return y

def genIfElse(x, y, z):
    return IfElse(x, y, z)

def genWhile():
    pass

def genDo(x):
    return x

def genWhileDo(t, x, y):
    return While(x, y)
